## ACTD 2024 - 20

### Clases 13-14: redes neuronales para clasificación

- Redes densas
- Redes para clasificación
- Procesamiento de datos continuos y categóricos
- Funciones de activación y pérdida

Empecemos importando numpy, pandas, keras, tensorflow

In [2]:
!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/385.2 MB ? eta -:--:--
    --------------------------------------- 8.9/385.2 MB 46.0 MB/s eta 0:00:09
   -- ------------------------------------- 19.4/385.2 MB 47.0 MB/s eta 0:00:08
   -- ------------------------------------- 28.0/385.2 MB 45.6 MB/s eta 0:00:08
   --- ------------------------------------ 38.0/385.2 MB 45.5 MB/s eta 0:00:08
   ----- ---------------------------------- 48.2/385.2 MB 46.5 MB/s eta 0:00:08
   ------ --------------------------------- 59.5/385.2 MB 46.8 MB/s eta 0:00:07
   ------- -------------------------------- 70.0/385.2 MB 47.5 MB/s eta 0:00:07
   -------- ------------------------------- 81.0/385.2 MB 47.4 MB/s eta 0:00:07
   --------- ------------------------------ 91.8/385.2 MB 48.0 MB/s eta 0:00:07
   ---------- ---------------------------- 101.4/385.2 MB 47.6 MB/s eta 0:00:06
   ----------- --------------------------- 112.5/385

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\saral\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

ModuleNotFoundError: No module named 'tensorflow'

Carguemos el archivo auto-mpg.data (disponible en Bloque Neón) usando pandas. Note que debemos incluir los nombres de las columnas, además de indicar los caracteres para separación, nas y comentarios.

In [ ]:
df = pd.read_csv('heart.csv')

Exploremos las primeras filas del dataframe.

In [ ]:
df.head()

Descripción de las variables:

https://archive.ics.uci.edu/dataset/45/heart+disease


Exploremos el tamaño del df

In [ ]:
df.shape

Identificamos NAs en los datos

In [ ]:
df.isna().sum()

Definimos listas para las variables categóricas enteras, categóricas string y numéricas.

In [ ]:
cat_int_feats = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca']

In [ ]:
cat_str_feats = ['thal']

In [ ]:
num_feats = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']

Agregamos las listas de categorías

In [ ]:
feats_ordered = cat_int_feats+cat_str_feats+num_feats

Reordenamos el dataframe de acuerdo con el tipo de variable

In [ ]:
df = df[feats_ordered+['target']]

In [ ]:
df.head()

Separamos los datos en entrenamiento, validación y prueba

In [ ]:
train = df.sample(frac=0.8, random_state=100)
train.head()

In [ ]:
train.shape

In [ ]:
test = df.drop(train.index)
test.head()

In [ ]:
val = train.sample(frac=0.2, random_state=100)

In [ ]:
val.shape

In [ ]:
train = train.drop(val.index)

In [ ]:
print(train.shape)
print(val.shape)
print(test.shape)

Calculamos estadísticas de cada variable numérica

In [ ]:
train.describe()

Función para convertir de dataframe (pandas) a dataset (tensorflow), separando características y etiquetas

In [ ]:
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

In [ ]:
train_ds = dataframe_to_dataset(train)
val_ds = dataframe_to_dataset(val)
test_ds = dataframe_to_dataset(test)

In [ ]:
type(train_ds)

Ejemplo de cómo queda el tf.dataset

In [ ]:
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

Separamos los datos de entrenamiento, validación y prueba en lotes

In [ ]:
batch_size = 32
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

Función para codificar variables numéricas (Keras docs)

In [ ]:
def encode_numerical_feature(feature, name, dataset):
    # Crea capa de normalización para este feature
    normalizer = keras.layers.Normalization()

    # Prepara el dataset para considerar únicamente la feature de interés (name)
    feature_ds = dataset.map(lambda x, y: x[name]) # selecciona variable
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1)) # deja el tensor de una dimensión

    # Aprende las estadísticas de los datos (media, varianza)
    normalizer.adapt(feature_ds)

    # Aplica la normalización a la variable
    encoded_feature = normalizer(feature)
    return encoded_feature

Función para codificar variables categóricas (Keras docs)

In [ ]:
def encode_categorical_feature(feature, name, dataset, is_string):
    lookup_class = keras.layers.StringLookup if is_string else keras.layers.IntegerLookup
    # Crea una capa Lookup para retornas variables 0/1 (dummies)
    # lookup: busca el valor correspondiente de la variable categórica
    lookup = lookup_class(output_mode="binary")

    # Prepara el dataset para considerar únicamente la feature de interés (name)
    feature_ds = dataset.map(lambda x, y: x[name]) # selecciona variable
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1)) # deja el tensor de una dimensión

    # Aprende el conjunto de posibles valores que toma la variable categórica y asigna enteros
    lookup.adapt(feature_ds)

    # Aplica la conversión de categorías a enteros
    encoded_feature = lookup(feature)
    return encoded_feature

Creamos una lista de inputs para el modelo, de acuerdo con cada tipo de variable

In [ ]:
inputs = []
for i in cat_int_feats:
  inputs.append(keras.Input(shape=(1,), name=i, dtype="int64"))

In [ ]:
for i in cat_str_feats:
  inputs.append(keras.Input(shape=(1,), name=i, dtype="string"))

In [ ]:
for i in num_feats:
  inputs.append(keras.Input(shape=(1,), name=i))

In [ ]:
for i in inputs:
   print(i)

Creamos una lista de variables codificadas/normalizadas de acuerdo con su tipo, empleando las funciones de codificación/normalización

In [ ]:
feats_encoded=[]

In [ ]:
for i,feat in enumerate(cat_int_feats):
  feats_encoded.append(
      encode_categorical_feature(inputs[i], feat, train_ds, False)
  )

In [ ]:
len_feats = len(feats_encoded)
len_feats

In [ ]:
for i,feat in enumerate(cat_str_feats):
  feats_encoded.append(
      encode_categorical_feature(inputs[len_feats+i], feat, train_ds, True)
  )

In [ ]:
len_feats = len(feats_encoded)
len_feats

In [ ]:
for i,feat in enumerate(num_feats):
  feats_encoded.append(
      encode_numerical_feature(inputs[len_feats+i], feat, train_ds)
  )

In [ ]:
for i in feats_encoded:
  print(i)

Creamos una capa concatenando todas las variables codificadas

In [ ]:
all_feats = keras.layers.concatenate(feats_encoded)

In [ ]:
type(all_feats)

Agregamos una capa densa con 32 neuronas y función de activación relu

In [ ]:
model_layers = keras.layers.Dense(32, activation='relu')(all_feats)

Agregamos la capa de salida con 1 neurona (probabilidad de sufrir la enfermedad cardiada) y función de activación sigmoide

In [ ]:
model_layers = keras.layers.Dense(1, activation='sigmoid')(model_layers)

Creamos el modelo con las capas ya creadas y las variables de entrada

In [ ]:
model = keras.Model(inputs, model_layers)

Compilamos el modelo, definiendo optimizador, función de pérdida y métricas adicionales a capturar

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

Aseguramos que Keras use TensorFlow como backend, para asegurar que el modelo pueda usar strings como entradas

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

Entrenamos el modelo con los datos en el formato tf.Dataset

In [ ]:
model.fit(train_ds, epochs=50, validation_data=val_ds)